In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

#### Import model, training function 

In [2]:
from annsa.load_dataset import dataset_to_spectrakeys
from annsa.model_classes import build_cnn_model, compile_model
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Dataset Construction

#### Load dataset

In [10]:
dataset = np.load(os.path.join('dataset_generation',
                               'uenrichment_dataset_100000.npy'))
training_spectra = np.array(dataset.item()['spectra'], dtype='float64')
all_keys = np.array(dataset.item()['keys'])

## Model Construction

In [3]:
architecture_id = os.path.join('..',
                               'source-interdiction',
                               'hyperparameter_search',
                               'hyperparameter-search-results',
                               'CNN-kfoldsfull-final-2-reluupdate_28',
                              )

In [20]:
model_features = load_features(architecture_id)
model_features.output_function = tf.nn.sigmoid
model_features.cnn_kernels = model_features.cnn_kernel
model_features.pool_sizes = model_features.pool_size
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.metrics = ['mse']
model_features.dropout_rate = model_features.dropout_probability
model_features.input_dim = 1024
model_features.output_size = 1

In [21]:
cae_model = load_model(
    os.path.join('..',
                 'source-interdiction',
                 'final_training_notebooks',
                 'final-models-keras',
                 'caepretrain-full-final.hdf5'))

## Train network

In [25]:
num_epochs = 200
patience = 10

model_id_save_as = 'caednn-full'

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    patience=patience,
    mode='min',
    min_delta=0.01,
    restore_best_weights=True)

## Scale input data

In [26]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)

# Train

In [27]:
kf = KFold(n_splits=5)

k = 0
for train_index, validation_index in kf.split(training_spectra_scaled):
    model = compile_model(
        build_cnn_model,
        model_features)
    model_weights = model.get_weights()
    model_weights_updated = model_weights[:]
    model_weights_updated[0:6] = cae_model.get_weights()[0:6]
    model.set_weights(model_weights_updated)

    csv_logger = tf.keras.callbacks.CSVLogger(
        os.path.join('final-models-keras/',
        model_id_save_as+'_fold' + str(k) + '.log')

    output = model.fit(
        x=training_spectra_scaled[train_index],
        y=all_keys[train_index],
        epochs=num_epochs,
        verbose=1,
        validation_data=(training_spectra_scaled[validation_index],
                         all_keys[validation_index]),
        shuffle=True,
        callbacks=[earlystop_callback, csv_logger],
    )
    model.save(
        os.path.join('final-models-keras',
                     model_id_save_as + '_fold' + str(k) + '.hdf5')
    k += 1

Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 27s 338us/sample - loss: 0.1172 - mean_squared_error: 0.0380 - val_loss: 0.0450 - val_mean_squared_error: 0.0284
Epoch 2/200
80000/80000 [==============================] - 24s 299us/sample - loss: 0.0339 - mean_squared_error: 0.0234 - val_loss: 0.0263 - val_mean_squared_error: 0.0195
Epoch 3/200
80000/80000 [==============================] - 24s 302us/sample - loss: 0.0253 - mean_squared_error: 0.0197 - val_loss: 0.0215 - val_mean_squared_error: 0.0168
Epoch 4/200
80000/80000 [==============================] - 24s 302us/sample - loss: 0.0221 - mean_squared_error: 0.0178 - val_loss: 0.0189 - val_mean_squared_error: 0.0152
Epoch 5/200
80000/80000 [==============================] - 24s 300us/sample - loss: 0.0200 - mean_squared_error: 0.0164 - val_loss: 0.0189 - val_mean_squared_error: 0.0156
Epoch 6/200
80000/80000 [==============================] - 24s 300us/sample - loss: 0.0187